In [1]:
import numpy as np
# Set parameters
sigma = 1
r_ = 0
drift = r_ - 0.5 * sigma**2
T = 100
m = 100
delta_t = T / m
N = 1000000 # number of simulations
# Number of RV we simulate is N * m
from scipy.special import zeta
B_1 = -zeta(0.5)/np.sqrt(2 * np.pi)

In [2]:
# Generate paths, note we skip time 0
def simProbMax0(drift, delta_t, sigma, N, m):
    sims = np.random.normal(drift * delta_t, sigma * delta_t**0.5, (N,m)) # note scale param is sd not var
    paths = np.cumsum(sims, axis=1)
    res = np.any(paths > 0, axis=1)
    return np.sum(~res) / N

prob = simProbMax0(drift, delta_t, sigma, N, m)
sd = (prob * (1 - prob) / N)**0.5
print(f"Probability of maximum 0 is about {prob}, with sd {sd}")
print(f"Hence 95% CI: [{prob - 1.96 * sd}, {prob + 1.96 * sd}")

Probability of maximum 0 is about 0.52952, with sd 0.0004991278088826548
Hence 95% CI: [0.52854170949459, 0.53049829050541


In [4]:
from scipy.special import zeta, factorial
#Solution of the probability using the theory of Johans paper thm 2.1 (not rewritten) redone later
def exactProbMax0InfTime(drift):
    B = - drift # this is beta
    r_values = np.arange(0, 150)
    zeta_vals = zeta(0.5 - r_values)
    summation = np.sum(zeta_vals/(factorial(r_values)*(2*r_values+1))* (- B**2 /2)**r_values)
    return np.exp(B / np.sqrt(2 * np.pi) * summation) * np.sqrt(2) * B

prob_exact = exactProbMax0InfTime(drift)
print(prob_exact)

0.5293251497992767


In [9]:
# Now we will estimate B2 of the paper Broadie 1999
# TODO research errors, sigma and T are fine, very high drift is also off
sigma = 0.5
T = 1
m = 2000
M = 1000000 # this is BM path, above is discrete
N = 1000

drift = 0.1 # Happens when r = 0.5 sigma^2
delta_t = T / M
def estB1B2():
    draws = np.random.normal(drift * delta_t, sigma * delta_t**0.5, (N, M))
    draws = np.c_[np.zeros(N), draws]
    BM_path = np.cumsum(draws, axis=1)
    max_bm = np.max(BM_path, axis=1)
    watch_moments = range(0, M, M//m)
    max_bw = np.max(BM_path[:, watch_moments], axis=1)
    z = BM_path[:, watch_moments]
    B_1_est = m**0.5 * np.mean(max_bm-max_bw)/(sigma * T**0.5)
    B_2_est = m*np.mean((max_bm - max_bw)**2)/(sigma**2 * T)
    return B_1_est, B_2_est
# print(np.sum((max_bw == max_bm))) # too much I think
# print(np.sum(((max_bw == max_bm) & (max_bm != 0)))) # too much I think
print("Beta 1 should be 0.5826 and Beta 2 0.425")
B_1_est, B_2_est = estB1B2()
print(f"Beta 1: {B_1_est}, ")#{np.std(max_bm- max_bw)/(N**0.5)}")
print(f"Beta 2: {B_2_est}")

Beta 1 should be 0.5826 and Beta 2 0.425
Beta 1: 0.5702285311519713, 
Beta 2: 0.43165433009653315


In [142]:
# Here we will use control variates and estimate the constant c
from scipy.special import zeta
B_1 = -zeta(0.5)/np.sqrt(2 * np.pi)
estimates = np.zeros((1500,2))
for i in range(1500):
    estimates[i] = estB1B2()

In [17]:
#Some estimates in finding B2
from scipy.special import zeta
B_1 = -zeta(0.5)/np.sqrt(2 * np.pi)
weirdConstant = zeta(-0.5) / np.sqrt(2 * np.pi)
print(B_1**2)
true_est = B_1**2 - 1/4
B_2 = 0.425
print("Covariance should be ", (B_2 - (B_1**2-1/4))*-1/2 )
print(1/true_est)
print(.425 / true_est)
print(0.425 - true_est)
print(1/(4 * B_1))

0.33941944843861255
Covariance should be  -0.16779027578069372
11.183249477170632
4.752881027797518
0.33558055156138744
0.4291129755668518


In [67]:
# Poging om covariance(M, Mhat-M) te vinden (partieel geslaagd, het zou kunnen)
drift = -0.0000004
m = 3000
M = 3000000 # this is BM path, above is discrete
N = 100
NumberOfSims = 100
max_bm = np.zeros(N * NumberOfSims)
max_bw = np.zeros(N*NumberOfSims)
for i in range(NumberOfSims):
    draws = np.random.normal(drift, (1/(M/m)**0.5), (N, M))
    draws = np.c_[np.zeros(N), draws]
    BM_path = np.cumsum(draws, axis=1)
    max_bm[i * N:(i+1)*N] = np.max(BM_path, axis=1)
    watch_moments = range(0, M, M//m)
    max_bw[i * N:(i+1)*N] = np.max(BM_path[:, watch_moments], axis=1)
print(np.cov(max_bm, max_bw - max_bm))

[[ 1.06741761e+03 -2.86294040e-01]
 [-2.86294040e-01  9.22640098e-02]]


In [162]:
covariances = np.cov(estimates[:,0], estimates[:,1])
c = -covariances[0,1]/covariances[0,0]
print(np.corrcoef(estimates[:,0], estimates[:,1]))

[[1.         0.92535239]
 [0.92535239 1.        ]]


In [161]:
# We have 92.5% correlation, which is very good for control variates
b1, b2 = estB1B2()
best_b2 = b2 + c * (b1 - B_1)




In [4]:
# Check formula for maximum geometric brownian motion on finite time
from scipy.stats import norm
sigma = 0.2
r_ = 0.1
drift = r_ - 0.5 * sigma**2
T = 1
m = 1000
delta_t = T / m
N = 100000 # number of simulations

def simMeanMaxGBM(drift, sigma, T, N, m):
    delta_t = T / m
    sims = np.random.normal(drift * delta_t, sigma * delta_t**0.5, (N,m)) # note scale param is sd not var
    paths = np.cumsum(sims, axis=1)
    max_bm = np.max(paths, axis=1)
    return np.mean(np.exp(max_bm))

mean_max_gbm = simMeanMaxGBM(drift, sigma, T, N, m)
print(mean_max_gbm)

def exactMeanMaxGBM(r_, sigma, T):
    return (1 + sigma**2 / (2*r_)) * np.exp(r_*T) * norm.cdf((r_ + 0.5 * sigma**2)*T/(sigma * T**0.5)) + \
        norm.cdf(- (r_ - 0.5 * sigma**2) * T / (sigma*T**0.5)) * (1 - sigma**2 / (2 * r_))

print(exactMeanMaxGBM(r_, sigma, T))

1.2334775106951026
1.238151824487769


In [5]:
# Prob of tildeM_m = 0 and hatM exact via Johan et al, we shall calculate both and compare
sigma = 0.1
r_ = -0.1
drift = r_ - 0.5 * sigma**2
T = 100
m = 5000
delta_t = T / m
N = 100000 # number of simulations

print(exactProbMax0InfTime(drift / sigma * np.sqrt(T / m)))
print(simProbMax0(drift, T/m, sigma, N, m))

0.19260500900844582
0.19259


In [6]:
# Here we will calculate an approximation for the mean of e^tilde(M)_m
S_0 = 100
approx_mean = S_0 * np.exp(- B_1 * sigma * np.sqrt(T / m)) * exactMeanMaxGBM(r_, sigma, T) + exactProbMax0InfTime(drift / sigma * np.sqrt(T / m)) * S_0
approx_price_option = np.exp(- r_ * T) * (approx_mean - S_0 * np.exp(r_ * T))
print("approx without full correction: ", np.exp(-r_ * T) * S_0 * np.exp(- B_1 * sigma * np.sqrt(T / m)) * exactMeanMaxGBM(r_, sigma, T) - S_0)
def simPriceLookbackPut(r_, sigma, T, N, m):
    drift = r_ - 0.5 * sigma ** 2
    mean_GBM = simMeanMaxGBM(drift, sigma, T, N, m)
    return np.exp(-r_ * T) * (mean_GBM * S_0 - S_0 * np.exp(r_ * T))

print(approx_price_option)
print(simPriceLookbackPut(r_, sigma, T, N, m))

approx without full correction:  2293701.819496535
2717942.583779832
2290308.5837396574


In [16]:
# Here we will numerically integrate my own idea, with the correction added later
from scipy.stats import norm
import scipy.integrate as integrate
def toIntegrate(x):
    def pdf(x):
        sqrtSigmaT = sigma * T**0.5
        return 1/sqrtSigmaT * norm.pdf((x - drift*T)/sqrtSigmaT) - 2 * drift / sigma**2 * np.exp(2 * drift*x / sigma**2) * norm.cdf((-x - drift * T)/sqrtSigmaT) +\
            np.exp(2 * drift * x / sigma**2) * norm.pdf((-x - drift * T)/sqrtSigmaT)/sqrtSigmaT
    def otherPdf(x):
        return 2 / (sigma * np.sqrt(2 * np.pi * T)) * np.exp(-1/(2 * sigma**2 * T) * (x-drift*T)**2) -\
            2 * drift/ sigma**2 * np.exp(2 * drift * x/ sigma**2)*norm.cdf((-x-drift*T)/(sigma * T**0.5))
    return x * pdf(np.log(x) + B_1 * sigma * np.sqrt(T/m) - np.log(S_0)) / x

# Some sanity tests
# Note pdf function ALSO integrates to one
# print(integrate.quad(toIntegrate, 0, np.inf)) integrates to one so density
# Both representations of pdfs are correct now
result = integrate.quad(toIntegrate, S_0, np.inf)[0]
print(np.exp(- r_ * T) * (result + exactProbMax0InfTime(drift / sigma * np.sqrt(T / m)) * S_0 - S_0 * np.exp(r_ * T)))
print("price without added prob 0", np.exp(- r_ * T) * (result - S_0 * np.exp(r_ * T)))
print(result)
print(exactMeanMaxGBM(r_, sigma, T) * S_0)
print(np.exp(-r_ * T) * (exactMeanMaxGBM(r_, sigma, T) * S_0 - S_0 * np.exp(r_*T)))

# Sanity check integrating toIntegrate but without the correction added over full support leads to
# the same mean as exactMeanMaxGbm (as it should), now it doesn't because correction is added (changing support is not sufficient)
# print(exactMeanMaxGBM(r_, sigma, T) * S_0, integrate.quad(toIntegrate, S_0, np.inf))

# Another check, the following results in the same price as approx without fully correction above (as expected)
# result = integrate.quad(toIntegrate, S_0 * np.exp(-B_1 * sigma * np.sqrt(T/m)), np.inf)[0]



2369462.2734250785
price without added prob 0 1945221.5091417814
88.3174598804879
105.0
2312678.9084547055


In [62]:
# Kansloze pogingen om B_2 te ontleden
gamma = -0.0000001
N = 100
m = 2000
M = m * 1000 # BM path is watched 100 times more than BW path
draws = np.random.normal(gamma, np.sqrt(1/(M/m)), (N, M))
draws = np.c_[np.zeros(N), draws]
BM_path = np.cumsum(draws, axis=1)
max_bm = np.max(BM_path, axis=1)
watch_moments = range(0, M, M//m)
max_bw = np.max(BM_path[:, watch_moments], axis=1)
print(np.mean((max_bm - max_bw)**2))

0.39067272328167396
[[ 6.56412421e+02 -2.91495324e-01]
 [-2.91495324e-01  8.65849801e-02]]
